In [1]:
import trimesh
import scipy
import numpy as np
import matplotlib.cm as cm
import nibabel as nib
import gdist
import sys
import pandas as pd
import importlib
from scipy import stats
import os

In [2]:
sys.path.insert(1, "/Users/dvarelat/Documents/MASTER/TFM/methods")
import cardiac_region
importlib.reload(cardiac_region)
import cardiac_region as c
sys.path.insert(1, "/Users/dvarelat/Documents/MASTER/TFM/ejemplo_Daniela_pintar_volumen")
import array2PLY

ModuleNotFoundError: No module named 'array2PLY'

In [ ]:
folder = "4_20190806_E3"
ESPECIMEN = folder.split("_")[1] + "_" + folder.split("_")[2]
file_mesh = (f"/Users/dvarelat/Documents/MASTER/TFM/midS/{ESPECIMEN}_myo_midS.ply")
folder_surface = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/SURFACE/{ESPECIMEN}/myo"
image_path = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/RESULTS/membranes/GASP_PNAS/{ESPECIMEN}_mGFP_XYZ_predictions_GASP.nii.gz"
DFFILE = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/{folder}/{ESPECIMEN}_cell_properties_radiomics.csv"

In [11]:
df_clean = pd.read_csv(DFFILE)
print(df_clean.shape)

(5216, 64)


In [12]:
d = 50
final_number_faces = 18000

In [14]:
proxy = nib.load(image_path)
img = np.asarray(proxy.get_fdata(), dtype=proxy.get_data_dtype())
print(img.shape)

(1024, 1024, 541)


In [15]:
dim_info = c.load3D_metadata(image_path)
mesh = trimesh.load_mesh(file_mesh, process=False)
mesh = mesh.simplify_quadratic_decimation(int(final_number_faces))
print(f"Numero de vertices {len(mesh.vertices)}")

Numero de vertices 9232


In [16]:
matrix = gdist.local_gdist_matrix(mesh.vertices, mesh.faces.astype(np.int32), d)
matrix_ij = scipy.sparse.find(matrix)
matrix_i, matrix_j, distances = matrix_ij[1], matrix_ij[0], matrix_ij[2]
j_coord = np.asarray(mesh.vertices[matrix_j])
len(j_coord)

3490425

In [39]:
j_pix = np.floor(j_coord / [dim_info["x_res"], dim_info["y_res"], dim_info["z_res"]]).astype("uint16")
j_VARIABLE_labels = img[j_pix[:, 0], j_pix[:, 1], j_pix[:, 2]]
unique_labels = set(j_VARIABLE_labels)
print(f"Unique labels {len(unique_labels)}")
print(len([i for i in unique_labels if i in list(df_clean.original_labels)]))

Unique labels 1112
943


In [52]:
tight_i = [np.where(matrix_i == i)[0][0] for i in np.unique(matrix_i)]
withme_i_VARIABLE = np.split(j_VARIABLE_labels, tight_i)[1:]
labels = list(df_clean.original_labels)

In [62]:
print(len(withme_i_VARIABLE))

VARIABLE_per_i = [len(set(v).intersection(labels)) for v in withme_i_VARIABLE]

9232


In [63]:
len(set(v).intersection(labels))

54

In [64]:
nombre = ESPECIMEN + "_" + "number" + "_" + str(d) + "myo.ply"  ## spl OR myo
file_out = os.path.join(folder_surface, nombre)
all_colors_median = np.zeros(shape=(mesh.vertices.shape[0], 4), dtype="float64")
colores_median = array2PLY.colorines(VARIABLE_per_i, cm.jet)
all_colors_median[np.unique(matrix_i)] = colores_median
mesh.visual.vertex_colors = all_colors_median
mesh.export(file_out)

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 9232\nproperty float x\nproperty float y\nproperty float z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nproperty uchar alpha\nelement face 18000\nproperty list uchar int vertex_indices\nend_header\n_\xea\nD\x9eraC\xf7|\x11B\xf8\xf5\x00\xff\x8c\xe5\xd0C\xa3\xadLC\xfc\xb6?C\xb4\xffC\xff\x11Z\xb4C\xe6\xcbZC\x1d\xcc\x07C\xce\xff)\xff\xd1\x82\xf1C&u\x83CM\xf8\x88B\xeb\xff\x0c\xff\xebn\xecC\xfa\xbe\x7fC\x8d-\xa0B\xff\xc4\x00\xff\xe6\xd8\x17D\x11\xefrC\xea\xfc1C\xb4\xffC\xff\xdb@\x80C\xee\xee\xa6COT\x06C\xff8\x00\xff\x0bi\xfcCf\xeeaC\xd0>\x17C}\xffz\xff&h\x04D\xeb\x05JC\xe0\x03\x90B\xff\x86\x00\xff\xaa\xad\x13D\xe2\xeb<C\xb1\xbb\x8aB\xc1\xff6\xff\xe8C7D]\r\xabC\xb6\x82\x11C\xff\xe6\x00\xff\x0e\xa6\x89C\x1e7|C\xb7\x14\x8fB\xff\xa7\x00\xff#\xea\xbdC:ZeC\x12J\x05C\xeb\xff\x0c\xff\xf6\x9b\x1dD[\x8c\xabCtx\xcdB\xc1\xff6\xff\xdd\xb1\xd2C\x99|fCS\x9bYB}\xffz\xff\x96]\x89CG~\x8eC\xa3\x

In [58]:
file_out

'/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/SURFACE/20190806_E3/myo/20190806_E3_number_50myo.ply'